# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [21]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [22]:
load_dotenv(override=True)
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')
openrouter_base_url = os.getenv('OPENROUTER_BASE_URL')

# Check the key

if not openrouter_api_key:
    print("No API key was found")
elif not openrouter_api_key.startswith("sk"):
    print("An API key was found, but it doesn't start with sk; please check you're using the right key")
else:
    print("API key found and looks good so far!")

# Check the base url

if not openrouter_base_url:
    print("No Base URL was found")
elif not openrouter_base_url.startswith("https://"):
    print("Base URL was found, but it doesn't start with https")
else:
    print("Base URL was found and looks good so far!")


API key found and looks good so far!
Base URL was found and looks good so far!


In [23]:

# constants

MODEL_GPT = 'gpt-oss-120b'   
MODEL_Claude = "anthropic/claude-3.5-sonnet"
openrouter = OpenAI(base_url='https://openrouter.ai/api/v1', api_key=openrouter_api_key)




In [24]:
import requests

def search_openlibrary(q: str) -> str:
    """
    Search Open Library for works matching query `q`.

    Returns a string that combines meaningful fields from the search results.
    """
    print("The book search library called:")
    base_url = "https://openlibrary.org/search.json"
    params = {"q": q}

    response = requests.get(base_url, params=params, timeout=10)
    response.raise_for_status()  # raises an error if HTTP 4xx/5xx

    data = response.json()

    total_results = data.get("numFound", 0)
    docs = data.get("docs", [])

    if not docs:
        return f"No results found for '{q}'. Total results reported: {total_results}."

    combined_results = [f"Query: {q}\nTotal Results: {total_results}\n"]

    for i, doc in enumerate(docs[:10], start=1):  # Limit to first 10 for readability
        title = doc.get("title", "N/A")
        authors = ", ".join(doc.get("author_name", ["Unknown"]))
        year = doc.get("first_publish_year", "Unknown")
        editions = doc.get("edition_count", "Unknown")
        languages = ", ".join(doc.get("language", [])) or "Unknown"
        ebook_access = doc.get("ebook_access", "Unknown")
        fulltext = doc.get("has_fulltext", False)

        combined_results.append(
            f"{i}. Title: {title}\n"
            f"   Authors: {authors}\n"
            f"   First Published: {year}\n"
            f"   Editions: {editions}\n"
            f"   Languages: {languages}\n"
            f"   eBook Access: {ebook_access}\n"
            f"   Full Text Available: {fulltext}\n"
        )

    return "\n".join(combined_results)

In [25]:
book_search_function = {
    "name": "search_openlibrary",
    "description": "Search for books on Open Library by a query string.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search term to find books (e.g., 'Harry Potter')",
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": book_search_function}]
tools


[{'type': 'function',
  'function': {'name': 'search_openlibrary',
   'description': 'Search for books on Open Library by a query string.',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': "The search term to find books (e.g., 'Harry Potter')"}},
    'required': ['query'],
    'additionalProperties': False}}}]

In [26]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        if tool_call.function.name == "search_openlibrary":
            arguments = json.loads(tool_call.function.arguments)
            q = arguments.get('query')
            book_details = search_openlibrary(q)
            responses.append({
                "role": "tool",
                "content": book_details,
                "tool_call_id": tool_call.id
            })
            print(f"Tool called: {tool_call.function.name} with args {arguments}")
    return responses

In [27]:
system_message = """
You are a helpful assistant to resolve technical issues and guide appropriately.
"""

In [28]:
history = []

In [ ]:
def stream_gpt(prompt):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": prompt}]
    
    # Stream GPT output
    stream = openrouter.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True,
        tools=tools
    )
    
    result = ""
    tool_calls_pending = False
    
    # Stream assistant text
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        result += delta
        yield delta 
        
        # Check if the model wants to call a tool
        if chunk.choices[0].finish_reason == "tool_calls":
            tool_calls_pending = True
            break  

    # Handle tool calls if needed
    if tool_calls_pending:
        response = openrouter.chat.completions.create(
            model=MODEL_GPT,
            messages=messages,
            tools=tools
        )
        
        while response.choices[0].finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_responses = handle_tool_calls(message) 
            
            # Stream tool outputs
            tool_text = ""
            for tool_output in tool_responses:
                content_str = tool_output["content"]
                if isinstance(content_str, dict):
                    content_str = json.dumps(content_str, indent=2) 
                tool_text += content_str
                yield content_str 
                
            # Update conversation
            messages.append(message)
            messages.extend(tool_responses)
            
            # Call the model again if tool suggests more actions
            response = openrouter.chat.completions.create(
                model=MODEL_GPT,
                messages=messages,
                #tools_metadata = tools_metadata
                tools=tools
            )
        
        # Append final tool-enhanced response to result
        final_text = response.choices[0].message.content or ""
        result += final_text
        if final_text:
            yield final_text 

    # Update conversation history
    history.append({"role": "user", "content": prompt})
    history.append({"role": "assistant", "content": result})

In [40]:
def stream_claude(prompt):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": prompt}]
    
    # Stream Claude output
    stream = openrouter.chat.completions.create(
        model=MODEL_Claude,
        messages=messages,
        stream=True,
        tools=tools
    )
    
    result = ""
    tool_calls_pending = False
    
    # Stream assistant text
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        result += delta
        yield delta 
        
        # Detect tool call
        if chunk.choices[0].finish_reason == "tool_calls":
            tool_calls_pending = True
            break  
    
    # Handle tool calls if requested
    if tool_calls_pending:
        response = openrouter.chat.completions.create(
            model=MODEL_Claude,
            messages=messages,
            tools=tools
        )
        
        while response.choices[0].finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_responses = handle_tool_calls(message) 
            
            # Stream tool outputs
            tool_text = ""
            for tool_output in tool_responses:
                content_str = tool_output["content"]
                if isinstance(content_str, dict):
                    content_str = json.dumps(content_str, indent=2)
                tool_text += content_str
                yield content_str  
            
            # Update conversation messages
            messages.append(message)
            messages.extend(tool_responses)
            
            # Call the model again if more tool calls are needed
            response = openrouter.chat.completions.create(
                model=MODEL_Claude,
                messages=messages,
                #tools_metadata = tools_metadata
                tools=tools
            )
        
        # Append final tool-enhanced text
        final_text = response.choices[0].message.content or ""
        result += final_text
        if final_text:
            yield final_text 
    
    # Update history
    history.append({"role": "user", "content": prompt})
    history.append({"role": "assistant", "content": result})

In [31]:

def stream_model(prompt, model):
    print(history)
   
    # Choose streaming generator
    if model == "GPT":
        result_stream = stream_gpt(prompt)
    elif model == "Claude":
        result_stream = stream_claude(prompt)
    else:
        raise ValueError("Unknown model")
    history.append({"role": "user", "content": prompt})
   
    assistant_content = ""
    for delta in result_stream:
        assistant_content += delta
        yield assistant_content
        

    # Store assistant output in history after streaming
    history.append({"role": "assistant", "content": assistant_content})




In [17]:
search_openlibrary('lords of the rings')

The book search library called:


'Query: lords of the rings\nTotal Results: 897\n\n1. Title: The Lord of the Rings\n   Authors: J.R.R. Tolkien\n   First Published: 1954\n   Editions: 251\n   Languages: pol, spa, glg, rus, cze, tur, ger, jpn, por, cat, fre, dut, eng, swe, slo, kor, ita, bul\n   eBook Access: borrowable\n   Full Text Available: True\n\n2. Title: The Two Towers\n   Authors: J.R.R. Tolkien\n   First Published: 1954\n   Editions: 278\n   Languages: jpn, baq, por, dan, ind, spa, dut, ast, rus, cze, tur, cat, fre, swe, afr, ita, pol, ger, hun, chi, eng, hrv, yid, bul\n   eBook Access: borrowable\n   Full Text Available: True\n\n3. Title: The Fellowship of the Ring\n   Authors: J.R.R. Tolkien\n   First Published: 1954\n   Editions: 297\n   Languages: jpn, por, dan, ind, spa, dut, ast, glg, rus, tur, cat, fre, swe, afr, ita, pol, ger, hun, chi, eng, hrv, yid, nor, bul\n   eBook Access: borrowable\n   Full Text Available: True\n\n4. Title: The Return of the King\n   Authors: J.R.R. Tolkien\n   First Published: 

In [37]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="Books", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Get me the details of the book harry potter.", "GPT"],
            ["Design an algorithm to find all strongly connected components in a directed graph.", "Claude"]
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


[{'role': 'user', 'content': 'Hi there'}, {'role': 'user', 'content': 'Hi there'}, {'role': 'assistant', 'content': 'Hello! How can I help you today?'}, {'role': 'assistant', 'content': 'Hello! How can I help you today?'}, {'role': 'user', 'content': 'Get me the details of the book harry potter.'}, {'role': 'user', 'content': 'Get me the details of the book harry potter.'}, {'role': 'assistant', 'content': 'Here are the top results for **“Harry Potter”** from the Open\u202fLibrary catalog. I’ve highlighted the most‑relevant titles (the seven main novels) and added a quick summary of the key information for each. If you’d like more details on any specific edition, language availability, or a particular format (e‑book, print‑disabled, etc.), just let me know!\n\n| # | Title (First Publication) | Author | Year | # of Editions | Languages (most common) | e‑Book Access | Full‑Text Available |\n|---|---------------------------|--------|------|--------------|------------------------|---------